#**Autoencoder Model for Word Embedding**

#I. Import necessary things

In [1]:
import tensorflow as tf
from tensorflow import keras
import re
import nltk
import numpy as np
from scipy.spatial import distance

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#II. Load corpus and preprocess it

In [2]:
corpus = """
BEIJING (AP) — China’s central bank on Friday declared all transactions involving Bitcoin and other virtual currencies illegal, stepping up a campaign to block use of unofficial digital money.

Friday’s notice complained Bitcoin, Ethereum and other digital currencies disrupt the financial system and are used in money-laundering and other crimes.

“Virtual currency derivative transactions are all illegal financial activities and are strictly prohibited,” the People’s Bank of China said on its website.

The price of Bitcoin fell more than 9%, to $41,085, in the hours after the announcement, as did most other crypto tokens. Ethereum skidded almost 10%, falling from $3,100 to around $2,800.

Chinese banks were banned from handling cryptocurrencies in 2013, but the government issued a reminder this year. That reflected official concern cryptocurrency mining and trading might still be going on or the state-run financial system might be indirectly exposed to risks.

Promoters of cryptocurrencies say they allow anonymity and flexibility, but Chinese regulators worry they might weaken the ruling Communist Party’s control over the financial system and say they might help to conceal criminal activity.

The People’s Bank of China is developing an electronic version of the country’s yuan for cashless transactions that can be tracked and controlled by Beijing.

Regulators in other countries have increasingly warned that cryptocurrencies need greater oversight. In the U.S., Gary Gensler, the chairman of the Securities and Exchange Commission, has said that investors need more protection in the cryptocurrency market, which he called “rife with fraud, scams and abuse” and compared to the “Wild West.”

The SEC has won dozens of cases against crypto fraudsters, but Gensler says the agency needs Congress to give it more authority and funding to adequately regulate the market.

Regulators in China have also been trying to rein in cryptocurrency mining, an energy-intensive process whereby specialized computers generate digital currencies. As a result, miners have been moving operations out of China.

Two years ago, China alone accounted for around three-quarters of all the electricity used for crypto mining, by far the most in the world, according to the Cambridge Bitcoin Electricity Consumption index. By April of this year, before the latest crackdown, China’s share had fallen back to 46%. That still towers over the No. 2 country, the United States, at less than 17%.



"""
print(corpus)


BEIJING (AP) — China’s central bank on Friday declared all transactions involving Bitcoin and other virtual currencies illegal, stepping up a campaign to block use of unofficial digital money.

Friday’s notice complained Bitcoin, Ethereum and other digital currencies disrupt the financial system and are used in money-laundering and other crimes.

“Virtual currency derivative transactions are all illegal financial activities and are strictly prohibited,” the People’s Bank of China said on its website.

The price of Bitcoin fell more than 9%, to $41,085, in the hours after the announcement, as did most other crypto tokens. Ethereum skidded almost 10%, falling from $3,100 to around $2,800.

Chinese banks were banned from handling cryptocurrencies in 2013, but the government issued a reminder this year. That reflected official concern cryptocurrency mining and trading might still be going on or the state-run financial system might be indirectly exposed to risks.

Promoters of cryptocurren

In [3]:
def text_cleaner(text):
    # lower case text
    text = text.lower()
    text = re.sub(r"'s\b","",text)
    # remove punctuations
    text = re.sub("[^a-zA-Z]", " ", text)
    return text

In [4]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

def preprocess_text(corpus):
  corpus_clean = text_cleaner(corpus)

  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(corpus_clean)
  
  return [w for w in word_tokens if not w in stop_words]

corpus_token =  preprocess_text(corpus)
print(corpus_token)

['beijing', 'ap', 'china', 'central', 'bank', 'friday', 'declared', 'transactions', 'involving', 'bitcoin', 'virtual', 'currencies', 'illegal', 'stepping', 'campaign', 'block', 'use', 'unofficial', 'digital', 'money', 'friday', 'notice', 'complained', 'bitcoin', 'ethereum', 'digital', 'currencies', 'disrupt', 'financial', 'system', 'used', 'money', 'laundering', 'crimes', 'virtual', 'currency', 'derivative', 'transactions', 'illegal', 'financial', 'activities', 'strictly', 'prohibited', 'people', 'bank', 'china', 'said', 'website', 'price', 'bitcoin', 'fell', 'hours', 'announcement', 'crypto', 'tokens', 'ethereum', 'skidded', 'almost', 'falling', 'around', 'chinese', 'banks', 'banned', 'handling', 'cryptocurrencies', 'government', 'issued', 'reminder', 'year', 'reflected', 'official', 'concern', 'cryptocurrency', 'mining', 'trading', 'might', 'still', 'going', 'state', 'run', 'financial', 'system', 'might', 'indirectly', 'exposed', 'risks', 'promoters', 'cryptocurrencies', 'say', 'allo

In [5]:
dictionary = sorted(list(set(corpus_token)))
mapping = dict((c, i) for i, c in enumerate(dictionary))
print(dictionary)

['abuse', 'according', 'accounted', 'activities', 'activity', 'adequately', 'agency', 'ago', 'allow', 'almost', 'alone', 'also', 'announcement', 'anonymity', 'ap', 'april', 'around', 'authority', 'back', 'bank', 'banks', 'banned', 'beijing', 'bitcoin', 'block', 'called', 'cambridge', 'campaign', 'cases', 'cashless', 'central', 'chairman', 'china', 'chinese', 'commission', 'communist', 'compared', 'complained', 'computers', 'conceal', 'concern', 'congress', 'consumption', 'control', 'controlled', 'countries', 'country', 'crackdown', 'crimes', 'criminal', 'crypto', 'cryptocurrencies', 'cryptocurrency', 'currencies', 'currency', 'declared', 'derivative', 'developing', 'digital', 'disrupt', 'dozens', 'electricity', 'electronic', 'energy', 'ethereum', 'exchange', 'exposed', 'fallen', 'falling', 'far', 'fell', 'financial', 'flexibility', 'fraud', 'fraudsters', 'friday', 'funding', 'gary', 'generate', 'gensler', 'give', 'going', 'government', 'greater', 'handling', 'help', 'hours', 'illegal',

#III. Convert corpus to one-hot vectors & Define embedding dim

In [6]:
embedding_dim = 20
vocab_size = len(dictionary)
print(vocab_size)

169


In [7]:
corpus_encode = [mapping[x] for x in corpus_token]
onehot_corpus = keras.utils.to_categorical(corpus_encode, num_classes=vocab_size)
print(onehot_corpus)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


#IV. Define Autoencoder model

In [8]:
ae_model = keras.Sequential()
ae_model.add(keras.Input(shape=(vocab_size,)))
ae_model.add(keras.layers.Dense(embedding_dim, activation='relu'))
ae_model.add(keras.layers.Dense(vocab_size, activation='softmax'))

ae_model.compile(optimizer='adam', loss='categorical_crossentropy')

ae_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                3400      
_________________________________________________________________
dense_1 (Dense)              (None, 169)               3549      
Total params: 6,949
Trainable params: 6,949
Non-trainable params: 0
_________________________________________________________________


#V. Train AE model

In [9]:
ae_model.fit(x=onehot_corpus, y=onehot_corpus, batch_size=32, epochs=1000)
ae_model.save("ae_model.h5")

Epoch 1/1000
7/7 [==============================] - 1s 2ms/step - loss: 5.1341
Epoch 2/1000
7/7 [==============================] - 0s 2ms/step - loss: 5.1242
Epoch 3/1000
7/7 [==============================] - 0s 2ms/step - loss: 5.1159
Epoch 4/1000
7/7 [==============================] - 0s 2ms/step - loss: 5.1074
Epoch 5/1000
7/7 [==============================] - 0s 2ms/step - loss: 5.0986
Epoch 6/1000
7/7 [==============================] - 0s 2ms/step - loss: 5.0894
Epoch 7/1000
7/7 [==============================] - 0s 3ms/step - loss: 5.0796
Epoch 8/1000
7/7 [==============================] - 0s 2ms/step - loss: 5.0690
Epoch 9/1000
7/7 [==============================] - 0s 3ms/step - loss: 5.0577
Epoch 10/1000
7/7 [==============================] - 0s 2ms/step - loss: 5.0452
Epoch 11/1000
7/7 [==============================] - 0s 2ms/step - loss: 5.0315
Epoch 12/1000
7/7 [==============================] - 0s 2ms/step - loss: 5.0169
Epoch 13/1000
7/7 [==============================

#=======================================================================

# **How to use trained model**

#=======================================================================

#VI. Load trained model & Get output of 1st FC layer

In [10]:
reconstructed_model = keras.models.load_model("ae_model.h5")
we_model = keras.models.Model(inputs=reconstructed_model.inputs, outputs=reconstructed_model.get_layer('dense').output)

Define function to encode one-hot list of words

In [11]:
def encode_onehot(mapping, list_words):
  output = []
  
  for word in list_words:
    word_vector = [0 for _ in range(vocab_size)]

    if word in mapping.keys():
      word_index = mapping[word]
      word_vector[word_index] = 1
    
    output.append(word_vector)
  
  return output

#VII. Test model

In [12]:
input_sentence = "Bitcoin is future money"

preprocess_sentence = preprocess_text(input_sentence)
onehot_sentence = encode_onehot(mapping, preprocess_sentence)

embedded_sentence = we_model.predict(onehot_sentence)
print(embedded_sentence)

[[1.7344453  0.         0.         0.         1.6993425  1.4995635
  1.6778412  0.10682005 1.6256998  0.01218796 1.4583051  1.5186117
  0.04581732 1.6100821  0.03771275 1.5422924  0.00361723 1.6116238
  0.09229034 1.6953753 ]
 [0.8174449  0.97222614 0.8464269  0.8151113  0.8595683  0.83100355
  0.81518614 0.86544025 0.90967464 0.8115623  0.7853561  0.86591923
  0.81248826 0.84176546 0.8474444  0.87158334 0.85262614 0.8532281
  0.81952477 0.84558386]
 [0.06684828 1.7982609  0.07724941 1.667414   1.4810903  1.6849318
  1.8128471  1.8991547  0.00594938 1.5764087  1.6640854  1.7199473
  1.5224851  1.5258919  2.0315151  0.         0.         2.0090332
  0.07331854 1.2025437 ]]


In [13]:
word_0 = "Bitcoin"
word_1 = "cryptocurrency"
word_2 = "Ethereum"
word_3 = "China"

preprocess_words = preprocess_text(' '.join([word_0, word_1, word_2, word_3]))
onehot_words = encode_onehot(mapping, preprocess_words)

word_0_eb, word_1_eb, word_2_eb, word_3_eb = we_model.predict(onehot_words)

print("OUTPUT EMBEDDING")
print(word_0_eb)
print(word_1_eb)
print(word_2_eb)
print(word_3_eb)
print("\n")

dst_1_3 = distance.euclidean(word_1_eb, word_0_eb)
dst_1_3 = distance.euclidean(word_2_eb, word_0_eb)

print("OUTPUT DISTANCE")

dst_0_1 = distance.euclidean(word_0_eb, word_1_eb)
print("Bitcoin vs cryptocurrency: ", dst_0_1)

dst_1_2 = distance.euclidean(word_1_eb, word_2_eb)
print("cryptocurrency vs Ethereum: ", dst_1_2)

dst_2_0 = distance.euclidean(word_2_eb, word_0_eb)
print("Ethereum vs Bitcoin: ", dst_2_0)

dst_0_3 = distance.euclidean(word_0_eb, word_0_eb)
print("Bitcoin vs China: ", dst_0_3)

dst_1_3 = distance.euclidean(word_1_eb, word_3_eb)
print("cryptocurrency vs China: ", dst_0_3)

dst_2_3 = distance.euclidean(word_1_eb, word_3_eb)
print("Ethereum vs China: ", dst_0_3)


OUTPUT EMBEDDING
[1.7344453  0.         0.         0.         1.6993425  1.4995635
 1.6778412  0.10682005 1.6256998  0.01218796 1.4583051  1.5186117
 0.04581732 1.6100821  0.03771275 1.5422924  0.00361723 1.6116238
 0.09229034 1.6953753 ]
[6.5165162e-03 2.5489229e-01 4.0286124e-02 7.7486038e-06 5.9490383e-02
 1.8225826e+00 0.0000000e+00 0.0000000e+00 1.6651645e+00 1.7698134e+00
 1.6463226e+00 1.6158335e+00 9.0993047e-03 6.3546121e-02 1.6737999e+00
 2.5362015e-02 1.6395812e+00 0.0000000e+00 1.8010042e+00 0.0000000e+00]
[1.65158451e-01 1.72169948e+00 1.74473512e+00 1.48164392e+00
 1.37150288e-04 1.65313506e+00 1.37985349e-02 1.18293405e-01
 1.78329539e+00 4.74304557e-02 9.20969248e-03 1.68192554e+00
 1.54734683e+00 1.62349558e+00 8.57170820e-02 6.31213188e-05
 4.48822975e-05 7.23381639e-02 1.69376504e+00 1.65728629e+00]
[1.3864665e+00 1.5625411e-01 1.4883863e+00 1.5828118e+00 2.3007393e-05
 1.4775991e-04 1.4660079e+00 8.3505511e-03 3.5234690e-03 1.3948932e+00
 1.5872958e+00 1.6790130e+00

Conslution